# Scraping Song Lyrics From Genius

NOTE: **Requires Outh2 authentication process**

In [1]:
from requests_oauthlib import OAuth2Session

In [2]:
import requests
from bs4 import BeautifulSoup
import json
import re
import numpy as np
import pandas as pd
import datetime
import os
import ast

In [3]:
client_id = 'hjAAh3f8SCTjC8YwuWOw_SC1Bbrld0-8pHzps79gZRexkQqJ1wYDHZCX8bWmMaQ3'
client_secret = 'WwCsfJyYc0VeAM200jBFnVRAAZME2RAeJemDd65XIjUTInpH4uSgW88sDcfI0YH5uoQtZYzUJWp7qkjcQQFoXA'

redirect_uri = 'https://genius.com/'

In [4]:
# base url
base_url = 'https://api.genius.com/'

auth_url = 'https://api.genius.com/oauth/authorize'

token_url = 'https://api.genius.com/oauth/token'

In [5]:
genius = OAuth2Session(client_id, redirect_uri=redirect_uri, scope='me', state='1')

# Redirect user to Genius for authorization
authorization_url, state = genius.authorization_url(auth_url)
print('Please go here and authorize,')
print(authorization_url)

Please go here and authorize,
https://api.genius.com/oauth/authorize?response_type=code&client_id=hjAAh3f8SCTjC8YwuWOw_SC1Bbrld0-8pHzps79gZRexkQqJ1wYDHZCX8bWmMaQ3&redirect_uri=https%3A%2F%2Fgenius.com%2F&scope=me&state=1


In [6]:
# PASTE THE RESPONSE URL HERE

# redirect_response = "#PASTE URL HERE"
redirect_response = "https://genius.com/?code=xEXhrpRNXwQzTNHPr9gyIrnnm6LnNRUQpz6ZNca6auqQdijyN1TH-SlwPi_btBLm&state=1"

In [7]:
# Fetch the access token
token_response = genius.fetch_token(token_url, client_secret=client_secret, authorization_response=redirect_response, client_id=client_id, response_type='code')

In [9]:
my_access_token = token_response['access_token']
# print(my_access_token)

In [10]:
# Set the API Headers
api_headers = {'Authorization': f'Bearer {my_access_token}'}

In [11]:
# # test search on Genius, given a TEST song id
# test_url = base_url + 'songs/378195'
# response = genius.get(test_url, headers=api_headers)

# print(response)

<Response [200]>


In [12]:
# Test search for a song/artist
# Response is same as typing "Toosie Slide Drake" into the search box on Genius.com
song_title = 'Toosie Slide'
artist_name = 'Drake'

query = (f'{song_title} {artist_name}').replace(" ", "%20")
search_query = f'search?q={query}'
search_url = base_url + search_query

r = genius.get(search_url, headers=api_headers)

In [13]:
search_api_path = r.json()['response']['hits'][0]['result']['api_path']

In [14]:
# using found song api path, get lyrics
song_url = f"https://api.genius.com{search_api_path}&access_token={my_access_token}"
# print(song_url)
r2 = genius.get(song_url, headers=api_headers)
# print(r2)

https://api.genius.com/songs/5402847&access_token=yDjel9-6lKQ8TXU9h47FLZwTnJDGPrq1MbjWWa_WXw_o0ohWKMIuvA7m6F0lZ2dd
<Response [200]>


In [15]:
song_json = r2.json()['response']['song']
lyrics_url = song_json['url']

In [16]:
# Scrape the actual song lyrics using BS4

page = requests.get(lyrics_url)
html = BeautifulSoup(page.text, "html.parser")
lyrics = html.find("div", class_="lyrics").get_text()

In [17]:
# Write the song's lyrics to a text file

fname = song_title.replace(' ', '') + '.txt'

with open(fname, 'wb') as f:
    lyrics_encoded = lyrics.encode('utf8')
    f.write(lyrics_encoded)

In [19]:
# Write the song's Genius.com metadat to json file
fname = song_title.replace(' ', '') + '.json'

with open(fname, 'w') as f:
    json.dump(song_json, f)